In [ ]:

import os
import sys
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from itertools import product
from sklearn.preprocessing import RobustScaler, LabelEncoder

sys.path.extend([
                '../src',
                '../data',
                ])

from utils import run_nested_cv_3b

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("CPU")
    
seed_num = 42
random.seed(seed_num)
np.random.seed(seed_num)
tf.random.set_seed(seed_num)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

#data
left = pd.read_csv('../data/left.csv')
right = pd.read_csv('../data/right.csv')
affected = pd.read_csv('../data/affected.csv')
unaffected = pd.read_csv('../data/unaffected.csv')

#limb
right['limb'] = 1
left['limb'] = 2
affected['limb'] = 3
unaffected['limb'] = 4

affected['subject'] = affected['subject'] + 10
unaffected['subject'] = unaffected['subject'] + 10

le = LabelEncoder()
right['use_type_encoded'] = le.fit_transform(right['use_type'])
left['use_type_encoded'] = le.fit_transform(left['use_type'])
affected['use_type_encoded'] = le.fit_transform(affected['use_type'])
unaffected['use_type_encoded'] = le.fit_transform(unaffected['use_type'])

combined = pd.concat([unaffected, 
                      affected, 
                      right, 
                      left], ignore_index=True)

architecture = {
                'num_filters': [50, 25],
                'filter_sizes': [5, 5],
                'pool_sizes': [5, 5]
               }

In [ ]:


approach = 3
filename = f'../results/approach3b/limb_nested_cv_left.csv'

window_size = 100
epochs = 200
batch_size = 32

class_loss = False
plot_hist = False

error_func = 'mae'

X = left[['ax', 'ay', 'az']]
y = left['gnd']
subjects = left['subject']
limbs = left['limb']
usetypes = left['use_type_encoded']

hp_grid = {
        'activation': ['relu', 'tanh'],
        'kernel_init': [tf.keras.initializers.HeNormal(seed=42), 
                        tf.keras.initializers.GlorotNormal(seed=42), ],
        'bias_init': [tf.keras.initializers.Zeros()],
        'optimizer':['adam', 'nadam'],
        'loss': [tf.keras.losses.mean_absolute_error
                 ,tf.keras.losses.mean_squared_error,
                 ],
        'learning_rate': [0.001, 0.0001],
        'batch_norm': [True, False]
        }

results_3b = run_nested_cv_3b(window_size,             
                              X=X,                         
                              y=y,                        
                              subjects=subjects,           
                              limbs=limbs,               
                              usetypes=usetypes,          
                              architecture=architecture,   
                              hp_grid=hp_grid,
                              epochs=epochs,                  
                              batch_size=batch_size,              
                              class_loss=False,           
                              plot_hist=False,            
                              error_func='mae')

print(np.mean(results_3b['youden_index']), np.std(results_3b['youden_index']))